In [24]:
import pandas as pd
import numpy as np
from sklearn.decomposition import TruncatedSVD,NMF
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from gensim import models, matutils

In [2]:
test = pd.read_csv('french_clean.csv')
test = test.clean

In [14]:
def display_topics(model, feature_names, no_top_words, topic_names=None):
    for ix, topic in enumerate(model.components_):
        if not topic_names or not topic_names[ix]:
            print("\nTopic ", ix)
        else:
            print("\nTopic: '",topic_names[ix],"'")
        print(", ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))
        
# words that don't help
stops = ['madeleine','ben','caroline','catherine','alexandre','germain','henry','ann','thérèse','marguerite',
         'mme','comte','comtesse','docteur','capitaine','duc','prince','miss','sir','sire','marquis','mademoiselle','baron',
         'colonel','abbé','oncle','reine','maman','curé','tante','cousin','papa','duchesse','baronne','indien','princesse',
         'lady','lord','mme','mmlle'
         'coeur','don','île','navire','hé','canadien','cardinal','chevalier','police','vicomte',
         'toutefois','alentour','encor','relativement',
         'répéta','répliqua']

In [15]:
tf = TfidfVectorizer(min_df=.05,max_df=.65,stop_words=stops)
tf_test = tf.fit_transform(test)

# LSA Model
Some okay topics, but overall pretty weak.

In [27]:
lsa = TruncatedSVD(10,random_state=8,n_iter=10)

In [28]:
doc_topic = lsa.fit_transform(tf_test)

In [29]:
display_topics(lsa,tf.get_feature_names(),10)


Topic  0
monseigneur, amant, prêtre, coeur, bureau, président, empereur, fusil, ministre, neige

Topic  1
monseigneur, gentilhomme, citoyen, mousquetaire, prisonnier, moine, épée, maréchal, empereur, royal

Topic  2
coeur, matelot, chasseur, fusil, rocher, canot, troupe, plaine, commandant, vaisseau

Topic  3
coeur, amant, soeur, poète, poésie, oeuvre, monseigneur, artiste, chère, vanité

Topic  4
citoyen, république, politique, national, révolution, gouvernement, patriote, nation, empereur, républicain

Topic  5
coeur, soeur, oeuvre, commissaire, agent, auto, inspecteur, assassin, président, directeur

Topic  6
matelot, chasseur, vaisseau, canot, guerrier, chaloupe, marin, gouverneur, commandant, pirate

Topic  7
monseigneur, poète, auto, temple, matelot, mousquetaire, chinois, mosquée, japonais, palais

Topic  8
matelot, notaire, marin, vaisseau, chaloupe, canot, équipage, bateau, bâtiment, etc

Topic  9
matelot, commandant, amant, empereur, peintre, marin, bal, loge, vaisseau, boul

# NMF model
Pretty decent topics (I went with the count vectorised in the end, however, because I felt its topics were more broadly representative).

In [20]:
nmf = NMF(random_state=8,shuffle=True,n_components=14)
doc_topic = nmf.fit_transform(tf_test)

In [21]:
display_topics(nmf,tf.get_feature_names(), 15)


Topic  0
atelier, café, trottoir, boulevard, bureau, gentil, gare, ventre, peintre, sale, patron, gamin, parisien, wagon, caresse

Topic  1
monseigneur, mousquetaire, gentilhomme, carrosse, épée, moine, régent, maréchal, éper, laquais, valet, cavalier, royal, dauphin, milord

Topic  2
loup, fermier, neige, vache, maire, paroisse, bourg, messe, prêtre, métayer, moulin, blé, pré, notaire, grange

Topic  3
notaire, rente, président, banquier, politique, amant, parisien, artiste, avoué, ministre, million, procureur, avocat, province, peintre

Topic  4
citoyen, république, patriote, national, marat, révolution, assemblée, président, républicain, convention, jacobin, prisonnier, royaliste, ministre, tuilerie

Topic  5
coeur, soeur, oeuvre, pèlerin, amant, volupté, harpe, palais, cadavre, royal, démon, blanche, huguenot, poète, violon

Topic  6
chasseur, guerrier, ours, bandit, troupe, camp, fusil, prairie, prisonnier, tribu, cavalier, rocher, pirate, moine, hutte

Topic  7
temple, mosquée, 

# LDA
This doesn't work well because my books are not based on topics for the most part - many of them will include multiple topics.

In [30]:
tft = tf_test.transpose()

In [33]:
corpus = matutils.Sparse2Corpus(tft)
id2word = dict((v, k) for k, v in tf.vocabulary_.items())

In [42]:
lda = models.LdaModel(corpus=corpus, num_topics=13, id2word=id2word, passes=100,random_state=8)
lda.print_topics()

[(0,
  '0.001*"coeur" + 0.001*"monseigneur" + 0.001*"prêtre" + 0.001*"amant" + 0.001*"bureau" + 0.001*"empereur" + 0.001*"fusil" + 0.001*"président" + 0.001*"prisonnier" + 0.001*"neige"'),
 (1,
  '0.000*"monseigneur" + 0.000*"poète" + 0.000*"claire" + 0.000*"fée" + 0.000*"nourrice" + 0.000*"ours" + 0.000*"atelier" + 0.000*"vallée" + 0.000*"marie" + 0.000*"demoiselle"'),
 (2,
  '0.000*"pascal" + 0.000*"poète" + 0.000*"vengeance" + 0.000*"empereur" + 0.000*"neige" + 0.000*"citoyen" + 0.000*"florin" + 0.000*"nation" + 0.000*"canon" + 0.000*"bonhomme"'),
 (3,
  '0.000*"monseigneur" + 0.000*"neige" + 0.000*"artiste" + 0.000*"marin" + 0.000*"rocher" + 0.000*"major" + 0.000*"publier" + 0.000*"lecteur" + 0.000*"chasseur" + 0.000*"bourse"'),
 (4,
  '0.000*"archer" + 0.000*"prêtre" + 0.000*"coeur" + 0.000*"monseigneur" + 0.000*"nation" + 0.000*"savant" + 0.000*"gendarme" + 0.000*"commandant" + 0.000*"dépit" + 0.000*"gentilhomme"'),
 (5,
  '0.000*"marat" + 0.000*"république" + 0.000*"matelot" + 0